In [1]:
# --- 0. Imports ---
from math import ceil
from helpers.strategy_helper import StrategyHelper

# Distance Based Race Strategy

The following notebook aims to determine different strategies for a distance based race such as The Sukuza 1000km.

The iRacing UI specifies the number of laps as 173.    This will be the number of laps the race leader will complete.  The remaining drivers will see the White Flag as the cross the line after the leader crosses to start their 173rd lap.  In other words following drivers may not complete 173 laps.

#### 1. Core Values are defined such as target_lap_time and tank_capacity

In [2]:
# Target Lap Time (s)
target_lap_time = 121.5

# Race Information
race_distance_km = 1000         # Total Race Length (km)
lap_length_km = 5.807           # Lap Length (km)  5.807 km (3.608 mi)

# Fuel Info
tank_capacity = 103.9           # Tank Capacity (litres)
fuel_per_avg = 3.57             # Basic Fuel/Lap  (litres)
fuel_per_max = 3.67
fuel_per_min = 3.46

# Pit Stop Insights
total_pit_time = 62             # iRaceInsight = 01:01.800
service_time = 42               # From Observation (s) iRaceInsight = 00:40.150
tyre_change_time = 21.0         # From Observation (s) iRaceInsight = 15.616

#### 2. Calculate some further values based on primary inputs

In [3]:
# Calculate the total number of laps.   UI = 173
total_laps = ceil(race_distance_km / lap_length_km)
print(f"Total laps: {total_laps}")

# Pit Lane Loss - not including service time (s)
pit_delta = total_pit_time - service_time
print(f"Pit Delta: {pit_delta}s")

# Refueling Rate (l/s)
refuelling_rate = tank_capacity / service_time
print(f"Refuelling Rate: {refuelling_rate:.2f} l/s")

# Free Tyre Stop Litres (Rounded Up)
tyre_change_litres = ceil(tyre_change_time * refuelling_rate)
tyre_change_stint = tyre_change_litres / fuel_per_avg
print(f"Tyre Change Litres: {tyre_change_litres:.2f}l = ~{tyre_change_stint:.2f} laps")

Total laps: 173
Pit Delta: 20s
Refuelling Rate: 2.47 l/s
Tyre Change Litres: 52.00l = ~14.57 laps


In [4]:
# Create an instance of strategy_helper where our useful maths functions live
strategy_helper = StrategyHelper(tank_capacity, total_laps)

## Full Race Lap Strategies
#### 3. Basic Strategy
Basic Strategy assumes that the team completes all 173 laps regardless of their finishing position.

In [5]:
print(f"Basic Strategy based on {total_laps} laps.")

# For each of the fuel/lap usage levels
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):

    # Get the number of laps completed when fuel use is as specified
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)

    # Get the stint plan
    s = strategy_helper.stint_plan(f, stint_laps)

    # print a summary of the strategy
    print(strategy_helper.format_strategy_summary(s))

Basic Strategy based on 173 laps.
fuel/lap: 3.46; laps/stint: 30; stints: 6; stops: 5; last_stint_laps: 23;
fuel/lap: 3.57; laps/stint: 29; stints: 6; stops: 5; last_stint_laps: 28;
fuel/lap: 3.67; laps/stint: 28; stints: 7; stops: 6; last_stint_laps: 5;


#### 4. Equal Stint Strategy

This strategy equalises the stints to avoid any 'splash and dash' at the end.

In [6]:
stint_array = strategy_helper.calculate_stint_plan(
    total_laps=total_laps,
    tank_capacity=tank_capacity,
    fuel_per_lap=fuel_per_max,      # use worst-case fuel burn
    tyre_change_litres=tyre_change_litres
)

print(f"Equal Stint Strategy based on {total_laps} laps and {stint_array['laps_for_free_tyres']} laps for a free tyre change.")

# Print out the stint results
print(strategy_helper.format_stint_array_results(stint_array["stint_laps"], fuel_per_avg))

Equal Stint Strategy based on 173 laps and 15 laps for a free tyre change.
Stint 1: 25 laps → 89.25 L
Stint 2: 25 laps → 89.25 L
Stint 3: 25 laps → 89.25 L
Stint 4: 25 laps → 89.25 L
Stint 5: 25 laps → 89.25 L
Stint 6: 24 laps → 85.68 L
Stint 7: 24 laps → 85.68 L


#### 5. Equalise Final Stints

This strategy equalises the final two stints to reduce the risk of a 'splash and dash' at the end but maximising stint length during the bulk of the race.

In [7]:
stint_laps = strategy_helper.stint_laps(fuel_per_max, tank_capacity)
print(f"Equalise Final Stints Strategy based on {total_laps} laps and {stint_array['laps_for_free_tyres']} Free Tyre laps.")

# Build a stint array based on maximum stints except last two which are equalised
stint_array = strategy_helper.build_last_two_even_from_full_stint(
    total_laps=total_laps,
    max_stint_laps=stint_laps,  # from fuel_per_max and tank_capacity
    fuel_per_lap=fuel_per_max,  # worst-case burn
    tyre_change_litres=tyre_change_litres,
    tank_capacity=tank_capacity,
)

# Print out the stint results
print(strategy_helper.format_stint_array_results(stint_array["stint_laps"], fuel_per_avg))

Equalise Final Stints Strategy based on 173 laps and 15 Free Tyre laps.
Stint 1: 28 laps → 99.96 L
Stint 2: 28 laps → 99.96 L
Stint 3: 28 laps → 99.96 L
Stint 4: 28 laps → 99.96 L
Stint 5: 28 laps → 99.96 L
Stint 6: 17 laps → 60.69 L
Stint 7: 16 laps → 57.12 L


## Adjusted Total Laps
#### 6.  Accounting For Leader Lap Pace in Total Lap Calculation

If we end up a lap down do we end once the leader finishes?

So if the leader crosses the line to start their 173 lap and then we cross behind them to start our 169 lap.  Will we only do 169 laps?

In [8]:
# Leaders Lap Pace
leader_pace = 119
race_time_seconds = total_laps * leader_pace
your_laps = int(race_time_seconds // target_lap_time)

print("Leader total laps:", total_laps)
print("Race time (s):", race_time_seconds)
print("Your completed laps:", your_laps)

Leader total laps: 173
Race time (s): 20587
Your completed laps: 169


#### 6b.  Checking our overal lap count based on Race Leader Performance and our Fuel Usage

In [9]:
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)
    your_laps_done = strategy_helper.simulate_laps_with_pits(
        total_race_laps=total_laps,
        leader_lap_time=leader_pace,
        leader_pit_loss=total_pit_time,
        leader_stint_laps=stint_laps,
        your_lap_time=target_lap_time,
        your_pit_loss=total_pit_time,
        your_stint_laps=stint_laps,
    )

    print(f"@ {f} l/lap: Leader completes {total_laps} vs {your_laps_done} laps for us")

@ 3.46 l/lap: Leader completes 173 vs 169 laps for us
@ 3.57 l/lap: Leader completes 173 vs 169 laps for us
@ 3.67 l/lap: Leader completes 173 vs 169 laps for us


#### 7. Adjusted Basic Strategy
Basic Strategy assumes that the team completes a reduced number of laps due to their finishing position.

In [10]:
print(f"Adjusted Basic Strategy based on {your_laps_done} laps.")
strategy_helper = StrategyHelper(tank_capacity, your_laps_done)

# For each of the fuel/lap usage levels
for f in (fuel_per_min, fuel_per_avg, fuel_per_max):

    # Get the number of laps completed when fuel use is as specified
    stint_laps = strategy_helper.stint_laps(f, tank_capacity)

    # Get the stint plan
    s = strategy_helper.stint_plan(f, stint_laps)

    # print a summary of the strategy
    print(strategy_helper.format_strategy_summary(s))

Adjusted Basic Strategy based on 169 laps.
fuel/lap: 3.46; laps/stint: 30; stints: 6; stops: 5; last_stint_laps: 19;
fuel/lap: 3.57; laps/stint: 29; stints: 6; stops: 5; last_stint_laps: 24;
fuel/lap: 3.67; laps/stint: 28; stints: 7; stops: 6; last_stint_laps: 1;


#### 8. Adjusted Equal Stint Strategy

Equal Stint Strategy adjust to account for running less laps than the leader


In [11]:
stint_array = strategy_helper.calculate_stint_plan(
    total_laps=your_laps_done,
    tank_capacity=tank_capacity,
    fuel_per_lap=fuel_per_max,  # use worst-case fuel burn
    tyre_change_litres=tyre_change_litres
)

# Print out summary messages and alerts
strategy_helper.print_fuel_limited_warning(stint_array)
if stint_array.get("status") != "fuel_limited":
    print("OK:", stint_array)

# Print out the stint results
print(strategy_helper.format_stint_array_results(stint_array["stint_laps"], fuel_per_avg))



⚠️  Fuel-Limited Stint Configuration Detected
--------------------------------------------------
One or more stints exceed tank capacity using this fuel_per_lap.

Maximum legal laps per stint at this burn: 28

Stints exceeding fuel capacity:
  • Stint 1: 29 laps (max 28) -> needs 1 lap(s) fuel saving

Suggestion: Use fuel_per_avg or reduce stint length (driver must fuel save).
--------------------------------------------------

Stint 1: 29 laps → 103.53 L
Stint 2: 28 laps → 99.96 L
Stint 3: 28 laps → 99.96 L
Stint 4: 28 laps → 99.96 L
Stint 5: 28 laps → 99.96 L
Stint 6: 28 laps → 99.96 L
